# Image Segmentation


This mainly will be using skimage methods and matplot and napari for viewing the images. 



## Import Lines

In [ ]:
import pandas as pd
import imageio as io
import numpy as np

import napari
from skimage import filters, morphology, measure, exposure, segmentation, restoration
import skimage.io as skio
import scipy.stats as st
from scipy import ndimage as ndi
from scipy.signal import find_peaks
from scipy import sparse

import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import plotly
import plotly.express as px
import plotly.graph_objects as go


## Reading in the Image

2019_Nov23_p117_mlsGFP_UASRFP_MFt5l_XY1574554994_Z000_T0_C2.tif

In [ ]:
C2 = skio.imread('2019_Nov23_p117_mlsGFP_UASRFP_MFt5l_XY1574554994_Z000_T0_C2.tif', plugin = 'tifffile')
z_size, x_size, y_size = C2.shape
z_scale = .5
xy_scale = 0.71
print('The mean of the TIFF stack is: ', C2.mean())
data = C2
print(f'number of dimensions: {C2.ndim}')
print(f'shape: {C2.shape}')
print(f'dtype: {C2.dtype}')

The mean of the TIFF stack is:  118.91694482097076
number of dimensions: 3
shape: (240, 704, 984)
dtype: uint16


Looking at the histogram of the image stack.

In [ ]:

fig, ax = plt.subplots(figsize=(8, 4))

ax.hist(C2.flatten(), log=True,
        bins=4096, range=(0, 4096))

_ = ax.set_title('Min value: %i \n'
                 'Max value: %i \n'
                 'Image shape: %s \n'
                 % (C2.min(),
                    C2.max(),
                    C2.shape))

## Napari Viewer

 For the sake of clear viewing, the "spacing" parameter will be set to .184 which is the nanometers for each step in the x and y positions.

In [ ]:
# z, y, x

spacing = np.array([.5,.184,.184])

## Exposure Adjustments

Comparing the following:

- Adaptive Histogram Equalization
- Rescale Intensity




In [ ]:
viewer_1 = napari.Viewer()

In [ ]:
viewer_1.add_image(
    data,
    scale = spacing,
)

### Rescale Intensity

To decide which will provide better results, we will continue with histogram equalization and rescale intensity as two of the possible ones to improve the image.

Rescale intensity returns an image after streching or shrinking its intensity levels. There are two parameters, in_range and out_range, which are used to stretch or shrink the intensity range of the input image. 
By default, the min/max intensities of the input image are stretched to the limits allowed by the image's dtype since in_range defauls to "image" and out_range defaults to "dtype".
Our dtype is uint16.

We will be using rescale intensity.

In [ ]:
rescaled_intensity = exposure.rescale_intensity(data)

In [ ]:
sigma_est = restoration.estimate_sigma(rescaled_intensity, average_sigmas=True)


In [ ]:
denoise = restoration.denoise_wavelet(rescaled_intensity, sigma = sigma_est,
                           method='BayesShrink', mode='soft',
                           rescale_sigma=True)


Comparing the histograms between the original image, the rescaled intensity image, and the adaptive histogram image.

In [ ]:

fig, (ax1,ax2,ax3) = plt.subplots(1,3, figsize=(16, 8))

bins = np.linspace(data.min(),data.max())

ax1.hist(data.flatten(), log=True,
        bins=bins, range=(data.min(), data.max()))

ax1.set_title('Original:\n' 'Min value: %i \n'
                 'Max value: %i \n'
                 'Image shape: %s \n'
                 % (data.min(),
                    data.max(),
                    data.shape))
ax2.hist(rescaled_intensity.flatten(), log=True,
        bins=bins, range=(data.min(), data.max()))

ax2.set_title('Rescaled Intensities \n' 'Min value: %i \n'
                 'Max value: %i \n'
                 'Image shape: %s \n'
                 % (rescaled_intensity.min(),
                    rescaled_intensity.max(),
                    rescaled_intensity.shape))

ax3.hist(data_adap.flatten())

ax3.set_title('Adaptive Histogram Equalization \n' 'Min value: %i \n'
                 'Max value: %i \n'
                 'Image shape: %s \n'
                 % (data_adap.min(),
                    data_adap.max(),
                    data_adap.shape))

### Filters

These filters will include different blurs such as Gaussian and Median (for now, will remove depening on if one is distinctly better than the other).

## Processing Steps, Morphological, and Segmentation

- Edge Detection with Sobel Filters
- Median Filter
- Multiotsu Threshold

**Morphological steps**
- Binary Closing
- Binary Opening
- Binary Dilation
- Binary Erosion

- Clear Border
- Distance Transform
- Smoothing of the distance transform with a gaussian blur
- Finding the local maxima with the smoothed distance transform
- Plotted points on the image


### Sobel filter

Finding edges in an image using the Sobel filter.
The edge filter is computed via the formula

sobel_mag = np.sqrt(sum(\[sobel(image, axis=i)**2
                         
                         for i in range(image.ndim)]) / image.ndim)

The magnitude is also computed if axis is a sequence.

In [ ]:
median = filters.median(denoise)


In [ ]:
edges_sobel = filters.sobel(median)



### Histogram Equalization

The Adaptive Histogram Equalization is a Contrast Limited Adaptive Histogram Equalization(CLAHE). This algorithm is for local contrast enhancements. It uses the histograms computed over different tile regions of the image. The local details can therefore be enhanced even in regions that are darker or lighter than most of the image. 

Some of the parameters included in this function includes the image, kernel_size, clip_limit and nbins.
The kernel size defines the shape of the contextual regions used in the algorithm. By default, the kernel_size is 1/8 of image height by 1/8th of its width.

In [ ]:
data_adap = exposure.equalize_adapthist(data)

#### Median Filter

Median filter smooths the image nicely and works better with the multiomask!!

Multiplying the median filter to the adaptive histogram image. The adaptive histogram removed some of the background that was lower in intensity compared to the rest of the image. 


In [ ]:
multiplied = edges_sobel * data_adap


Using an unsharp filter to remove (or sharpen) more the edges that can get loss with a median filter.

In [ ]:
unsharp = filters.unsharp_mask(multiplied,radius = 2, amount = 1)

Multiplying the unsharp image and the skeleton mask that was created earlier. The skeleton image was dilated 10 (for a total of 14 as the skeleton was dilated in ImageJ 4 times prior to being loaded into this program) times so there is a slightly bigger area for the image to look at.

In [ ]:
skeleton = skio.imread('comp1Skel.tif', plugin = 'tifffile')

for i in range(0,10):
    skeleton = morphology.dilation(skeleton)
    i +=1

med_skel = filters.median(skeleton)


In [ ]:
ROI = unsharp * med_skel

### MultiOtsu's Threshold







#### MultiOtsu

Return threshold value based on Otsu’s method.

Either image or hist must be provided. If hist is provided, the actual histogram of the image is ignored.

Using this on the ROI.

In [ ]:
multiotsu = filters.threshold_multiotsu(ROI,2)

multiomask = ROI > multiotsu

#Morphological Steps to Improve the Mask

CODE: Closing, Opening, Dilation, Erosion steps
These provide cleaner results for the mask and picks up key parts of the mask that were otherwise missed. 
Information on these steps can be found [here](https://github.com/scikit-image/scikit-image/blob/main/skimage/morphology/binary.py#L121-L153).

**Binary Closing**: Returns a fast binary morphological closing of an image. Binary closing consists of dilation followed by erosion of the image. Closing is intended to remove small dark spots and connect small bright cracks. This performs faster on binary images (provided from the initial multiotsu mask) than grayscale closing.

**Binary Opening**: Erosion followed by dilation. 

**Binary Dilation**: Dilation enlarges bright regions and shrinks dark regions. 

**Binary Erosion**: Shrinks bright regions and enlarges dark regions

In [ ]:
multiotsu_closing = morphology.binary_closing(multiomask)
multiotsu_opening = morphology.binary_opening(multiotsu_closing)
multiotsu_dilation = morphology.binary_dilation(multiotsu_opening)
multiotsu_CODE = morphology.binary_erosion(multiotsu_dilation)

In [ ]:
cleared = segmentation.clear_border(multiotsu_CODE)
label_image = measure.label(cleared)
boundaries = segmentation.find_boundaries(label_image)
viewer_1.add_labels(
    boundaries,
    scale=spacing,
)

In [ ]:
# adding the points to the image at the specific locations in the mask

transformed = ndi.distance_transform_edt(multiotsu_CODE,sampling=spacing)
smooth_distance = filters.gaussian(transformed)

maxima = morphology.local_maxima(smooth_distance)
labeled_maxima = measure.label(maxima, connectivity = maxima.ndim)

viewer_1.add_points(
    np.transpose(np.nonzero(labeled_maxima)),
    name = 'test points',
    scale=spacing,
    size=4,
    n_dimensional=True,
)

### Some analysis

Main measurements we want:
- Intensity
- Coordinates
- Length

In [ ]:
#regions  = measure.regionprops(label_image =labeled_maxima, intensity_image = rescaled_intensity) #labels is our labeled image, imstack is
# the intensity image

properties = ['label', 'area', 'bbox_area', 'bbox', 'mean_intensity', 'equivalent_diameter',
'minor_axis_length', 'major_axis_length', 'centroid']



props = measure.regionprops_table(label_image,intensity_image = C2, properties = properties)
df = pd.DataFrame(props).set_index('label')
df


,area,bbox_area,bbox-0,bbox-1,bbox-2,bbox-3,bbox-4,bbox-5,max_intensity,min_intensity,mean_intensity,equivalent_diameter,minor_axis_length,major_axis_length,centroid-0,centroid-1,centroid-2
label,,,,,,,,,,,,,,,,,
1,4165,129532,31,362,221,78,415,273,935,216,444.190636,19.962065,17.705918,88.755440,50.401200,387.879232,243.460984
2,7,27,52,364,248,55,367,251,396,341,367.000000,2.373376,3.023716,3.023716,53.000000,365.000000,249.000000
3,629,3876,73,345,271,85,364,288,552,245,375.627981,10.630426,11.471760,22.505420,78.289348,353.836248,278.157393
4,911,4199,82,336,287,99,349,306,567,225,374.643249,12.027441,11.749617,23.599280,89.578485,341.781559,296.809001
5,12,36,115,333,346,118,337,349,413,293,351.750000,2.840496,3.265986,3.829708,116.000000,334.500000,347.000000
6,117,336,125,353,390,131,361,397,494,246,352.119658,6.068235,7.434016,8.833626,127.444444,356.427350,392.760684
7,401,1386,127,375,416,138,384,430,624,199,356.541147,9.149175,10.868343,13.463056,131.780549,378.960100,422.224439
8,7,27,131,368,420,134,371,423,358,248,303.857143,2.373376,3.023716,3.023716,132.000000,369.000000,421.000000
9,7,27,137,389,441,140,392,444,351,201,278.285714,2.373376,3.023716,3.023716,138.000000,390.000000,442.000000
